In [ ]:
import json
import pandas as pd
import json
from tqdm import tqdm
import pandas as pd
import tiktoken
import numpy as np

from openai import OpenAI
from credentials import OPENAI_API_KEY

client = OpenAI(api_key=OPENAI_API_KEY)
dataset_path = ""

In [ ]:
# Load dataset
df = pd.read_csv(dataset_path)
df

,Text,Label,Tokens,embedding
0,Budget to set scene for election\n \n Gordon B...,0,648,"[0.021319035440683365, 0.008336679078638554, 0..."
1,Army chiefs in regiments decision\n \n Militar...,0,584,"[0.015592621639370918, -0.015502342022955418, ..."
2,Howard denies split over ID cards\n \n Michael...,0,638,"[0.02563329227268696, 0.0022945322562009096, 0..."
3,Observers to monitor UK election\n \n Minister...,0,598,"[0.027909917756915092, 0.07791844010353088, 0...."
4,Kilroy names election seat target\n \n Ex-chat...,0,597,"[-0.03326362371444702, -0.006136978976428509, ..."
...,...,...,...,...
2220,India opens skies to competition\n \n India wi...,4,267,"[-0.006999113131314516, -0.010065499693155289,..."
2221,Yukos bankruptcy 'not US matter'\n \n Russian ...,4,507,"[-0.056246452033519745, 0.01597324199974537, 0..."
2222,Survey confirms property slowdown\n \n Governm...,4,423,"[0.039799414575099945, 0.04613218829035759, 0...."
2223,High fuel prices hit BA's profits\n \n British...,4,572,"[-0.002154754940420389, 0.002955003408715129, ..."


In [ ]:
# Estimate token cost
enc = tiktoken.encoding_for_model("text-embedding-3-small")
df["Tokens"] = df["Text"].apply(lambda x: len(enc.encode(x)))
df.to_csv(dataset_path, index=False)

print("Total tokens:", df["Tokens"].sum())

Total tokens: 1078178


In [ ]:
# Generate embeddings if not exists
if "embedding" not in df.columns:

    batch_size = 100
    all_embeddings = []

    for i in tqdm(range(0, len(df), batch_size), desc="Embedding batches"):
        batch_texts = df["Text"].iloc[i:i+batch_size].tolist()
        response = client.embeddings.create(
            model="text-embedding-3-small",
            input=batch_texts
        )
        batch_embeddings = [item.embedding for item in response.data]
        all_embeddings.extend(batch_embeddings)

    df["embedding"] = [json.dumps(vec) for vec in all_embeddings]
    df.to_csv(dataset_path, index=False)

df

,Text,Label,Tokens,embedding
0,Budget to set scene for election\n \n Gordon B...,0,648,"[0.021319035440683365, 0.008336679078638554, 0..."
1,Army chiefs in regiments decision\n \n Militar...,0,584,"[0.015592621639370918, -0.015502342022955418, ..."
2,Howard denies split over ID cards\n \n Michael...,0,638,"[0.02563329227268696, 0.0022945322562009096, 0..."
3,Observers to monitor UK election\n \n Minister...,0,598,"[0.027909917756915092, 0.07791844010353088, 0...."
4,Kilroy names election seat target\n \n Ex-chat...,0,597,"[-0.03326362371444702, -0.006136978976428509, ..."
...,...,...,...,...
2220,India opens skies to competition\n \n India wi...,4,267,"[-0.006999113131314516, -0.010065499693155289,..."
2221,Yukos bankruptcy 'not US matter'\n \n Russian ...,4,507,"[-0.056246452033519745, 0.01597324199974537, 0..."
2222,Survey confirms property slowdown\n \n Governm...,4,423,"[0.039799414575099945, 0.04613218829035759, 0...."
2223,High fuel prices hit BA's profits\n \n British...,4,572,"[-0.002154754940420389, 0.002955003408715129, ..."


In [33]:
taxonomy_ids = {
    0: "Politics",
    1: "Sport",
    2: "Technology",
    3: "Entertainment",
    4: "Business"
}

labels = list(taxonomy_ids.values())

prompt = f"""
For each of the following categories, produce a short, mutually exclusive description.
Keep each description 1–2 sentences max.
Return the result strictly as a JSON object mapping each label to its description.
Categories: {labels}
"""

resp = client.chat.completions.create(
    model="gpt-4o-mini",
    temperature=0,
    messages=[{"role": "user", "content": prompt}],
    max_tokens=500,
    response_format={"type": "json_object"}
)

taxonomy_desc = json.loads(resp.choices[0].message.content)

taxonomy = {
    idx: {
        "label": label,
        "description": taxonomy_desc[label]
    }
    for idx, label in taxonomy_ids.items()
}

print(json.dumps(taxonomy, indent=4))

{
    "0": {
        "label": "Politics",
        "description": "The process by which groups of people make collective decisions, often involving governance, policy-making, and the exercise of power."
    },
    "1": {
        "label": "Sport",
        "description": "Physical activities and games that involve competition, skill, and often teamwork, typically governed by a set of rules."
    },
    "2": {
        "label": "Technology",
        "description": "The application of scientific knowledge for practical purposes, encompassing tools, machines, and systems that enhance human capabilities."
    },
    "3": {
        "label": "Entertainment",
        "description": "Activities designed to provide enjoyment, amusement, or pleasure, including films, music, theater, and games."
    },
    "4": {
        "label": "Business",
        "description": "The organized effort to produce, sell, or provide goods and services for profit, involving various economic activities and management pra

In [34]:
taxonomy = {}
for idx, label in tqdm(taxonomy_ids.items()):
    desc = taxonomy_desc[label]
    resp = client.embeddings.create(
        model="text-embedding-3-small",
        input=desc
    )
    taxonomy[idx] = {
        "label": label,
        "description": desc,
        "embedding": resp.data[0].embedding
    }

with open("taxonomy_with_embeddings.json", "w") as f:
    json.dump(taxonomy, f, indent=4)

preview = {
    idx: {
        "label": data["label"],
        "description": data["description"],
        "embedding": data["embedding"][:5] + ["..."]
    }
    for idx, data in taxonomy.items()
}
print(json.dumps(preview, indent=2))

100%|██████████| 5/5 [00:03<00:00,  1.52it/s]

{
  "0": {
    "label": "Politics",
    "description": "The process by which groups of people make collective decisions, often involving governance, policy-making, and the exercise of power.",
    "embedding": [
      0.04627108573913574,
      0.01733066327869892,
      0.03743255138397217,
      0.040329743176698685,
      0.015483178198337555,
      "..."
    ]
  },
  "1": {
    "label": "Sport",
    "description": "Physical activities and games that involve competition, skill, and often teamwork, typically governed by a set of rules.",
    "embedding": [
      0.01830044388771057,
      0.026326123625040054,
      0.04467391595244408,
      0.013766762800514698,
      0.032386813312768936,
      "..."
    ]
  },
  "2": {
    "label": "Technology",
    "description": "The application of scientific knowledge for practical purposes, encompassing tools, machines, and systems that enhance human capabilities.",
    "embedding": [
      0.03167610988020897,
      0.02624591812491417,
    

In [35]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.metrics.pairwise import cosine_similarity

# Load doc embeddings
df = pd.read_csv(dataset_path)
df["embedding"] = df["embedding"].apply(json.loads)
X = np.vstack(df["embedding"].to_list())

# Load taxonomy & embeddings
with open("taxonomy_with_embeddings.json") as f:
    taxonomy = json.load(f)

taxonomy = {int(k): v for k, v in taxonomy.items()}
tax_ids = sorted(taxonomy.keys())  # [0,1,2,3,4]
tax_labels = [taxonomy[i]["label"] for i in tax_ids]
tax_vectors = np.vstack([taxonomy[i]["embedding"] for i in tax_ids])

# Classification with cosine similarity
cos_sim = cosine_similarity(X, tax_vectors)
pred_ids = np.argmax(cos_sim, axis=1)

df["predicted_label_id"] = pred_ids
df["predicted_label_name"] = [tax_labels[i] for i in pred_ids]

# Evaluation
y_true = df["Label"]
y_pred = df["predicted_label_id"]

print("Accuracy:", accuracy_score(y_true, y_pred))
print("Precision:", precision_score(y_true, y_pred, average="macro"))
print("Recall:", recall_score(y_true, y_pred, average="macro"))
print("F1:", f1_score(y_true, y_pred, average="macro"))

print("\nResults:")
print(classification_report(y_true, y_pred, target_names=tax_labels))

Accuracy: 0.7595505617977528
Precision: 0.760220302718861
Recall: 0.7550525903446814
F1: 0.7507016566555117

Results:
               precision    recall  f1-score   support

     Politics       0.74      0.83      0.78       417
        Sport       0.80      0.85      0.82       511
   Technology       0.78      0.54      0.64       401
Entertainment       0.66      0.81      0.73       386
     Business       0.83      0.75      0.79       510

     accuracy                           0.76      2225
    macro avg       0.76      0.76      0.75      2225
 weighted avg       0.77      0.76      0.76      2225



In [38]:
df

,Text,Label,Tokens,embedding,predicted_label_id,predicted_label_name
0,Budget to set scene for election\n \n Gordon B...,0,648,"[0.021319035440683365, 0.008336679078638554, 0...",4,Business
1,Army chiefs in regiments decision\n \n Militar...,0,584,"[0.015592621639370918, -0.015502342022955418, ...",0,Politics
2,Howard denies split over ID cards\n \n Michael...,0,638,"[0.02563329227268696, 0.0022945322562009096, 0...",0,Politics
3,Observers to monitor UK election\n \n Minister...,0,598,"[0.027909917756915092, 0.07791844010353088, 0....",0,Politics
4,Kilroy names election seat target\n \n Ex-chat...,0,597,"[-0.03326362371444702, -0.006136978976428509, ...",4,Business
...,...,...,...,...,...,...
2220,India opens skies to competition\n \n India wi...,4,267,"[-0.006999113131314516, -0.010065499693155289,...",4,Business
2221,Yukos bankruptcy 'not US matter'\n \n Russian ...,4,507,"[-0.056246452033519745, 0.01597324199974537, 0...",4,Business
2222,Survey confirms property slowdown\n \n Governm...,4,423,"[0.039799414575099945, 0.04613218829035759, 0....",4,Business
2223,High fuel prices hit BA's profits\n \n British...,4,572,"[-0.002154754940420389, 0.002955003408715129, ...",4,Business
